setting up a neural network regression using Keras with a dataset that is stored in a JSON file as a list of objects:

In [12]:
import json
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split

# Load the data from the JSON file
with open('user_input/scenarios.json') as f:
    data = json.load(f)

# Extract the features and labels from the data
X = []
y = []
for example in data:
    X.append([
        example['weather'],
        example['vehicle'],
        example['traffic'],
        example['timeOfDay'],
        example['location'],
        example['intersections'],
        example['route_length'],

    ])
    y.append(example['total_difficulty_rating'])

# Load the data from the JSON file
with open('user_input/scenarios.json') as f:
    data = json.load(f)


# One-hot encode the features
from sklearn.preprocessing import OneHotEncoder

# Create a list of indices corresponding to the categorical columns in X
cat_cols = [0, 1, 2, 3, 4, 5]

# Initialize the one-hot encoder
encoder = OneHotEncoder(categories='auto', sparse=False)

# Fit the encoder on X
encoder.fit(X)

# Transform X using the encoder
X_encoded = encoder.transform(X)

# Print the shape of the encoded X
print(X_encoded.shape)

# Convert the data to NumPy arrays
X = X_encoded 
y = np.array(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the neural network model
model = Sequential()
model.add(Dense(16, input_shape=(30,), activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(4, activation='relu'))
model.add(Dense(2, activation='linear'))

# Compile the model
model.compile(loss='mean_squared_error', optimizer='adam')

# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model
loss = model.evaluate(X_test, y_test)
print('Test loss:', loss)

# Make predictions with the model
predictions = model.predict(X_test)


(20, 30)
Epoch 1/100
1/1 [==============================] - 2s 2s/step - loss: 60777.9609 - val_loss: 111990.1094
Epoch 2/100
1/1 [==============================] - 0s 60ms/step - loss: 60776.7266 - val_loss: 111988.3828
Epoch 3/100
1/1 [==============================] - 0s 53ms/step - loss: 60775.5703 - val_loss: 111986.6797
Epoch 4/100
1/1 [==============================] - 0s 53ms/step - loss: 60774.4648 - val_loss: 111985.0391
Epoch 5/100
1/1 [==============================] - 0s 50ms/step - loss: 60773.3672 - val_loss: 111983.5156
Epoch 6/100
1/1 [==============================] - 0s 56ms/step - loss: 60772.2812 - val_loss: 111981.9844
Epoch 7/100
1/1 [==============================] - 0s 59ms/step - loss: 60771.2500 - val_loss: 111980.4766
Epoch 8/100
1/1 [==============================] - 0s 48ms/step - loss: 60770.2656 - val_loss: 111978.9922
Epoch 9/100
1/1 [==============================] - 0s 60ms/step - loss: 60769.3164 - val_loss: 111977.5312
Epoch 10/100
1/1 [============

To generate new scenarios using the predictions from the neural network, you can use the predicted values as input for the parameters of the new scenarios. Here's an example of how you can do that:

Get the predicted values for each scenario from the neural network model.
Scale the predicted values back to their original range if you normalized the data during training.
Use the predicted values to generate new scenarios with higher difficulty scores. For example, you can increase the number of cars or pedestrians, or set the weather to a more challenging condition.
Save the new scenarios to a JSON file.

Here's an example of how you can generate new scenarios based on the predicted values:


In [8]:
import json

# # Load the predicted values from the neural network model
# with open('predicted_values.json', 'r') as f:
#     predicted_values = json.load(f)

predicted_values = predictions
# Scale the predicted values back to their original range if needed

# Generate new scenarios based on the predicted values
new_scenarios = []
for i in range(len(predicted_values)):
    # Increase the number of cars and pedestrians based on the predicted values
    num_cars = int(predicted_values[i][0] * 10) # Scale the value from 0-1 to 0-10
    pedestrians = True if predicted_values[i][1] > 0.5 else False # Set pedestrians to True if the value is above 0.5
    # Set other parameters to a fixed value or use the predicted values if appropriate
    scenario = {
        'weather': 'Rain',
        'road': 'City',
        'vehicle': 'Car',
        'traffic': 'Heavy',
        'emergency': 'No',
        'timeOfDay': 'Day',
        'location': 'Urban',
        "num_cars": num_cars,
        "intersection": True,
        "pedestrians": pedestrians,
        "pedestrian_cross": True,
    }
    new_scenarios.append(scenario)

# Save the new scenarios to a JSON file
with open('new_scenarios.json', 'w') as f:
    json.dump(new_scenarios, f, indent=4)


In [13]:
# Define a function to generate new scenarios based on the predictions
def generate_scenarios(model, encoder, n=10):
    # Create a list to store the generated scenarios
    scenarios = []
    
    # Generate n scenarios
    for i in range(n):
        # Create a random input vector
        input_vec = np.random.rand(1, 7)
        
        # One-hot encode the input vector
        input_vec_encoded = encoder.transform(input_vec)
        
        # Make a prediction with the model
        prediction = model.predict(input_vec_encoded)
        
        # Convert the prediction to a difficulty rating
        difficulty_rating = prediction[0][0]
        
        # Create a new scenario dictionary
        scenario = {
            'weather': 'sunny',
            'vehicle': 'car',
            'traffic': 'light',
            'timeOfDay': 'day',
            'location': 'urban',
            'intersections': 2,
            'route_length': 5.0,
            'total_difficulty_rating': difficulty_rating
        }
        
        # Add the scenario to the list
        scenarios.append(scenario)
    
    return scenarios

# Generate 10 new scenarios
new_scenarios = generate_scenarios(model, encoder, n=10)

# Print the new scenarios
for scenario in new_scenarios:
    print(scenario)


TypeError: ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''